In [56]:
import pandas as pd
from scipy.stats import pointbiserialr

data_train=pd.read_csv('data/data_train.csv')
data_input=pd.read_csv('data/test.csv')

data_result={'id':[],'price_range':[]}
correlation={}
data_min= {}
data_max= {}
columns=data_train.columns
non_numeric = ["blue","dual_sim","four_g","three_g","touch_screen","wifi"]

Korelasi

In [57]:
def find_correlation(corr,data_min,data_max,columns):
    for i in columns:
        if(i in non_numeric):
            correlation, p_value = pointbiserialr(data_train[i], data_train['price_range'])
            corr[i]=correlation
        else :
            corr[i]=data_train[i].corr(data_train['price_range'])
    for i in columns:
        data_min[i]=data_train[i].min()
    for i in columns:
        data_max[i]=data_train[i].max()

In [58]:
find_correlation(correlation,data_min,data_max,columns)

In [59]:
print(correlation)
print(data_min)
print(data_max)

{'battery_power': 0.18480092449553084, 'blue': 0.04194688860741449, 'clock_speed': 0.014031254818008083, 'dual_sim': -0.01075616373764093, 'fc': -0.003842010298191734, 'four_g': 0.0005508484718003666, 'int_memory': 0.026175706877841595, 'm_dep': 0.0012049180209846337, 'mobile_wt': -0.07476875048323661, 'n_cores': -0.0005823306285452805, 'pc': -0.005214430491652989, 'px_height': 0.15883273548307963, 'px_width': 0.1787126901102656, 'ram': 0.9183192307843839, 'sc_h': 0.012148883173074988, 'sc_w': 0.019911698810365006, 'talk_time': 0.011112731754754877, 'three_g': 0.027097628312175107, 'touch_screen': -0.029842480103712064, 'wifi': 0.03432935762117285, 'price_range': 0.9999999999999998}
{'battery_power': 501, 'blue': 0, 'clock_speed': 0.5, 'dual_sim': 0, 'fc': 0, 'four_g': 0, 'int_memory': 2, 'm_dep': 0.1, 'mobile_wt': 80, 'n_cores': 1, 'pc': 0, 'px_height': 0, 'px_width': 500, 'ram': 256, 'sc_h': 5, 'sc_w': 0, 'talk_time': 2, 'three_g': 0, 'touch_screen': 0, 'wifi': 0, 'price_range': 0}
{

Normalisasi

In [60]:
def normalize(data,data_min,data_max,correlation,columns):
    for i in columns:
        if(i!='price_range'):
            data[i]=10*abs(correlation[i])*(data[i]-data_min[i])/(data_max[i]-data_min[i])

In [61]:
normalize(data_train,data_min,data_max,correlation,columns)
normalize(data_input,data_min,data_max,correlation,columns)

In [62]:
def distance(data1, data2, row1, row2):
    res = 0
    for i in columns:
        if i != 'price_range':
            res += (data1.loc[row1, i] - data2.loc[row2, i]) ** 2
    return res ** (1/2)


Classify Point

In [69]:
def classify_point(input_data,train_data,k,point) :
    arr = []
    for i in range(len(train_data)) :
        arr.append([distance(input_data, train_data, point,i),train_data.loc[i,'price_range']])
    arr.sort(key=lambda x: x[0])
    arr = arr[:k]
    count = [0,0,0,0]
    for i in arr :
        count[i[1]]+=1
    # print(count.index(max(count)))
    return count.index(max(count))

def accuracy(data_input,data_train,k) :
    count = 0
    for i in range(len(data_input)) :
        if(classify_point(data_input,data_train,k,i)==data_input.loc[i,'price_range']) :
            count+=1
    return count/len(data_input)


In [76]:
for i in range(3,20):
    print(i,accuracy(data_train[:50],data_train,i))

3 0.94
4 0.94
5 0.94
6 0.92
7 0.94
8 0.94
9 0.94
10 0.94
11 0.94
12 0.94
13 0.96
14 0.94
15 0.94
16 0.94
17 0.94
18 0.94
19 0.96


In [64]:
def iterate_k(input_data,train_data,result_data,k) :
    for i in range(len(input_data)):
        result_data['id'].append(i)
        result_data['price_range'].append(classify_point(input_data, train_data, k, i))

In [65]:
def write_output(result_data) :
    result_data.to_csv('data/result.csv',index=False)

In [66]:
iterate_k(data_input,data_train,data_result,23)

0
3
3
2
0
0
2
3
0
2
1
3
3
1
2
2
1
1
0
2
0
2
1
0
0
1
2
2
3
0
0
2
3
2
3
3
1
2
2
3
0
1
3
1
3
2
1
3
1
1
3
1
2
2
3
3
1
1
1
0
0
2
2
0
2
1
2
2
2
0
1
3
0
3
3
3
0
3
3
0
0
1
3
0
2
2
3
2
3
1
0
0
3
3
2
2
3
0
0
0
2
2
3
1
2
2
1
2
1
1
2
1
1
0
0
0
2
0
1
2
3
1
1
1
2
3
0
3
3
3
3
0
3
2
1
0
0
1
0
3
0
1
2
2
1
2
1
0
1
3
1
1
0
1
3
2
3
1
3
3
3
0
0
0
0
0
2
1
2
1
3
0
3
0
1
2
3
2
1
0
0
1
0
3
1
2
0
3
2
2
1
3
3
1
2
0
2
3
2
1
0
1
3
2
1
2
1
1
3
2
2
3
1
1
1
0
1
0
0
2
1
2
0
3
1
0
0
2
2
0
1
0
1
3
2
3
1
1
3
1
1
0
1
2
0
2
2
2
2
0
2
0
3
3
3
0
2
1
2
3
0
3
2
0
3
1
3
1
1
1
3
0
1
3
1
0
1
1
2
2
3
0
1
3
0
2
2
0
2
0
0
0
1
2
3
2
3
0
1
2
1
0
0
3
0
2
0
2
2
3
2
1
0
3
3
1
0
2
3
0
2
2
3
0
2
2
0
3
1
3
2
0
2
2
3
1
3
3
1
1
2
3
3
1
3
0
3
0
1
1
0
0
2
0
3
3
3
3
2
0
2
1
1
2
1
0
0
3
0
3
0
1
3
1
1
2
1
1
2
0
3
2
2
1
3
1
2
3
0
0
2
1
3
0
2
3
2
3
1
0
3
3
1
3
2
2
2
0
3
3
1
3
1
0
2
3
2
1
1
3
2
0
0
2
2
0
1
2
2
3
1
1
3
0
2
3
3
2
1
2
1
1
2
2
2
3
1
3
2
3
1
0
2
0
1
1
3
0
3
3
1
0
1
1
2
1
0
0
1
2
2
0
0
2
3
0
2
1
3
0
1
1
1
1
0
1
1
1
2
1
1
1
1
1
0
3
3
1
0
2
